<a href="https://colab.research.google.com/github/Krishnaa-tech/ML-Projects/blob/main/Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!git clone https://github.com/pjreddie/darknet

fatal: destination path 'darknet' already exists and is not an empty directory.


In [2]:
cd darknet

/content/darknet


In [3]:
ls

cfg/       include/      LICENSE.gen   LICENSE.mit  python/    src/
data/      LICENSE       LICENSE.gpl   LICENSE.v1   README.md
examples/  LICENSE.fuck  LICENSE.meta  Makefile     scripts/


In [4]:
!make

mkdir -p obj
mkdir -p backup
mkdir -p results
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -c ./src/gemm.c -o obj/gemm.o
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -c ./src/utils.c -o obj/utils.o
In file included from /usr/include/string.h:495,
                 from ./src/utils.c:3:
In function ‘strncpy’,
    inlined from ‘copy_string’ at ./src/utils.c:426:5:
/usr/include/x86_64-linux-gnu/bits/string_fortified.h:106:10: warning: ‘__builtin_strncpy’ specified bound depends on the length of the source argument [-Wstringop-overflow=]
  106 |   return __builtin___strncpy_chk (__dest, __src, __len, __bos (__dest));
      |          ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
./src/utils.c: In function ‘copy_string’:
./src/utils.c:426:22: note: length computed here
  426 |     strncpy(copy, s, strlen(s)+1);
      |                      ^~~~~~~~~
gcc -Iinclude/ -Isrc/ -Wa

In [5]:
!wget https://pjreddie.com/media/files/yolov3.weights

--2023-04-29 12:24:32--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M   101MB/s    in 2.3s    

2023-04-29 12:24:35 (101 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
!cp -r '/content/drive/My Drive/ML_Data/data1_1' '/content/'

In [8]:
!cp -r '/content/drive/My Drive/ML_Data/data2_2' '/content/'

^C


In [9]:
!./darknet detect cfg/yolov3.cfg yolov3.weights "data/data1_1/img1_1 10086.jpg"

layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   608 x 608 x   3   ->   608 x 608 x  32  0.639 BFLOPs
    1 conv     64  3 x 3 / 2   608 x 608 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    2 conv     32  1 x 1 / 1   304 x 304 x  64   ->   304 x 304 x  32  0.379 BFLOPs
    3 conv     64  3 x 3 / 1   304 x 304 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    4 res    1                 304 x 304 x  64   ->   304 x 304 x  64
    5 conv    128  3 x 3 / 2   304 x 304 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    6 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64  0.379 BFLOPs
    7 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    8 res    5                 152 x 152 x 128   ->   152 x 152 x 128
    9 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64  0.379 BFLOPs
   10 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
   11 res    8                 152 x 

In [12]:
import os
import cv2
import numpy as np

# set the paths for Darknet
darknet_path = "/darknet"
cfg_path = "/darknet/yolov3.cfg"
weights_path = "/darknet/yolov3.weights"
class_path = "/darknet/data/coco.names"

# load the network and class names
net = cv2.dnn.readNetFromDarknet(cfg_path, weights_path)
classes = open(class_path).read().strip().split("\n")

# set the paths for input and output folders
input_folder_path = "/content/darknet/data/data1_1"
output_folder_path = "/content/drive/MyDrive/ML_Data/Model_Predictions"

# create the output folder
os.makedirs(output_folder_path, exist_ok=True)

# iterate through all the files in the input folder
for filename in os.listdir(input_folder_path):

    # check if the file is an image file
    if filename.endswith(".jpg") or filename.endswith(".png"):

        # read the image using cv2.imread()
        img = cv2.imread(os.path.join(input_folder_path, filename))

        # create a blob from the image
        blob = cv2.dnn.blobFromImage(img, 1/255.0, (416,416), swapRB=True, crop=False)

        # set the input blob and perform a forward pass
        net.setInput(blob)
        layer_names = net.getLayerNames()
        output_layers = [layer_names[i[0]-1] for i in net.getUnconnectedOutLayers()]
        outputs = net.forward(output_layers)

        # loop over the outputs and extract the class and confidence of each detection
        for detection in outputs:
            for obj in detection:
                scores = obj[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]

                # check if the confidence is greater than the threshold
                if confidence > 0.5:

                    # get the coordinates of the bounding box
                    center_x = int(obj[0] * img.shape[1])
                    center_y = int(obj[1] * img.shape[0])
                    w = int(obj[2] * img.shape[1])
                    h = int(obj[3] * img.shape[0])
                    x = center_x - w//2
                    y = center_y - h//2

                    # draw the bounding box and label on the image
                    color = (0,255,0)
                    cv2.rectangle(img, (x,y), (x+w,y+h), color, 2)
                    label = "{}: {:.2f}%".format(classes[class_id], confidence*100)
                    cv2.putText(img, label, (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # save the new image to the output folder
        cv2.imwrite(os.path.join(output_folder_path, filename), img)

# print a message to indicate that all images have been saved
print("All images saved to output folder")


error: ignored

In [ ]:
import os
import cv2
import numpy as np

# set the paths for Darknet
darknet_path = "path/to/darknet"
cfg_path = "path/to/yolov3.cfg"
weights_path = "path/to/yolov3.weights"
class_path = "path/to/coco.names"

# load the network and class names
net = cv2.dnn.readNetFromDarknet(cfg_path, weights_path)
classes = open(class_path).read().strip().split("\n")

# set the paths for input and output folders
input_folder_path = "path/to/input/folder/"
output_folder_path = "path/to/output/folder/"

# create the output folder
os.makedirs(output_folder_path, exist_ok=True)

# iterate through all the files in the input folder
for filename in os.listdir(input_folder_path):

    # check if the file is an image file
    if filename.endswith(".jpg") or filename.endswith(".png"):

        # read the image using cv2.imread()
        img = cv2.imread(os.path.join(input_folder_path, filename))

        # create a blob from the image
        blob = cv2.dnn.blobFromImage(img, 1/255.0, (416,416), swapRB=True, crop=False)

        # set the input blob and perform a forward pass
        net.setInput(blob)
        layer_names = net.getLayerNames()
        output_layers = [layer_names[i[0]-1] for i in net.getUnconnectedOutLayers()]
        outputs = net.forward(output_layers)

        # initialize a list to store the bounding box coordinates and class names
        boxes = []

        # loop over the outputs and extract the class and confidence of each detection
        for detection in outputs:
            for obj in detection:
                scores = obj[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]

                # check if the confidence is greater than the threshold
                if confidence > 0.5:

                    # get the coordinates of the bounding box
                    center_x = int(obj[0] * img.shape[1])
                    center_y = int(obj[1] * img.shape[0])
                    w = int(obj[2] * img.shape[1])
                    h = int(obj[3] * img.shape[0])
                    x = center_x - w//2
                    y = center_y - h//2

                    # add the bounding box coordinates and class name to the list
                    boxes.append((x,y,w,h,classes[class_id]))

        # draw the bounding boxes and class labels on the image
        for box in boxes:
            cv2.rectangle(img, (box[0],box[1]), (box[0]+box[2],box[1]+box[3]), (0,255,0), 2)
            cv2.putText(img, box[4], (box[0],box[1]-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)

        # save the annotated image as a new file
        cv2.imwrite(os.path.join(output_folder_path, filename), img)

# print a message to indicate that all images have been saved
print("All images saved to output folder")


# COLOR DETECTION

In [ ]:
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
img_path = 'data/data1_1/img1_1 10062.jpg'
img = cv2.imread(os.path.join(img_path,'img1_1 10062.jpg'))
plt.imshow(img)
plt.show()

In [ ]:
image = plt.imread('data/data1_1/img1_1 10062.jpg')
plt.imshow(image)

In [ ]:
hsv = cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
plt.imshow(hsv)
plt.show()

In [ ]:
print(image.shape)

**CHAT GPT VERSION** 

In [ ]:
import cv2
import numpy as np

# Load the image
image = cv2.imread("img1_1 10062.jpg")

# Convert BGR image to HSV
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Define range of blue color in HSV
lower_blue = np.array([110,50,50])
upper_blue = np.array([130,255,255])

# Threshold the HSV image to get only blue colors
mask = cv2.inRange(hsv_image, lower_blue, upper_blue)

# Bitwise-AND mask and original image
result = cv2.bitwise_and(image, image, mask=mask)

# Display the result
cv2.imshow('Result', result)
cv2.waitKey(0)
cv2.destroyAllWindows()
